In [ ]:
#Python libraries

In [ ]:
import geopandas as gpd
import pandas as pd
import folium
from shapely.wkt import loads
from IPython.display import display

#This Python script performs spatial analysis to estimate and visualize the elderly population (65+) distribution within buildings in selected statistical subsections (BGRI zones) in Porto, Portugal

In [ ]:
# Load the data
bgri = gpd.read_file("BGRI2021_1312.gpkg")
edificios = pd.read_csv("edificios.csv")

# Convert buildings to a GeoDataFrame, ensuring the geometry column is correct
edificios['geometry'] = edificios['geometry'].apply(loads)
edificios = gpd.GeoDataFrame(edificios, geometry='geometry')

# Define CRS
edificios.set_crs(epsg=4326, inplace=True)
edificios = edificios.to_crs(epsg=3763)
bgri = bgri.to_crs(epsg=3763)

# Remove the 'index_right' column if it exists
if 'index_right' in edificios.columns:
    edificios.drop(columns=['index_right'], inplace=True)
if 'index_right' in bgri.columns:
    bgri.drop(columns=['index_right'], inplace=True)

# Fix potential topological errors in geometries
edificios['geometry'] = edificios['geometry'].buffer(0)
bgri['geometry'] = bgri['geometry'].buffer(0)

# Perform a spatial join to associate each building with a statistical subsection
edificios = gpd.sjoin(edificios, bgri[['SUBSECCAO', 'geometry']], how='left', predicate='intersects')

# Merge building data with BGRI population data
edificios = edificios.merge(bgri[['SUBSECCAO', 'N_INDIVIDUOS_65_OU_MAIS']], on='SUBSECCAO', how='left')

# Calculate the total area of buildings within each Statistical Subsection
edificios['area'] = edificios['geometry'].area
soma_areas = edificios.groupby('SUBSECCAO')['area'].sum().reset_index()
soma_areas.rename(columns={'area': 'TOTAL_AREA_SE'}, inplace=True)

# Merge with the buildings dataframe
edificios = edificios.merge(soma_areas, on='SUBSECCAO', how='left')

# Distribute the population proportionally to the building area using integer values
edificios['POP_65+_PROPORCIONAL'] = (
    (edificios['area'] / edificios['TOTAL_AREA_SE']) * edificios['N_INDIVIDUOS_65_OU_MAIS']
)
edificios['POP_65+_ARREDONDADA'] = edificios['POP_65+_PROPORCIONAL'].round()

# Ensure the sum matches the total elderly population
ajustes = (
    edificios.groupby('SUBSECCAO')['POP_65+_ARREDONDADA'].sum() - 
    edificios.groupby('SUBSECCAO')['N_INDIVIDUOS_65_OU_MAIS'].first()
)
ajustes = ajustes.reset_index()
ajustes.rename(columns={0: 'AJUSTE'}, inplace=True)

for _, row in ajustes.iterrows():
    subsecao = row['SUBSECCAO']
    diff = row['AJUSTE']  # Access the column correctly
    if diff != 0:
        subset = edificios[edificios['SUBSECCAO'] == subsecao]
        if not subset.empty:
            idx = subset.index[0]
            edificios.loc[idx, 'POP_65+_ARREDONDADA'] -= diff

# Fill null values
edificios['POP_65+_ARREDONDADA'] = edificios['POP_65+_ARREDONDADA'].fillna(0)

# Convert coordinates to latitude and longitude for Folium visualization
edificios = edificios.to_crs(epsg=4326)
bgri = bgri.to_crs(epsg=4326)

# Calculate the total 65+ population per subsection
pop_bgri = (
    edificios.groupby('SUBSECCAO')['POP_65+_ARREDONDADA']
    .sum()
    .reset_index()
    .rename(columns={'POP_65+_ARREDONDADA': 'POP_TOTAL_65+'})
)

# Merge the data with the BGRI GeoDataFrame
bgri = bgri.merge(pop_bgri, on='SUBSECCAO', how='left')
bgri['POP_TOTAL_65+'] = bgri['POP_TOTAL_65+'].fillna(0)

# Define the map center
centro = [41.14961, -8.61099]  # Approximate coordinates of Porto city center

# Create a Folium map centered on Porto
m = folium.Map(location=centro, zoom_start=14)

# Add building polygons to the map
for _, row in edificios.iterrows():
    color = 'red' if row['POP_65+_ARREDONDADA'] > 10 else 'blue'
    folium.GeoJson(
        row['geometry'],
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': color,
            'weight': 1,
            'fillOpacity': 0.6
        },
        tooltip=(
            f"Building {row['osm_id']}<br>Population 65+: {row['POP_65+_ARREDONDADA']}"
        )
    ).add_to(m)

# Add the BGRI subsections polygons to the map with transparency and tooltips showing the total 65+ population
folium.GeoJson(
    bgri,
    style_function=lambda x: {
        'fillColor': 'green',
        'color': 'green',
        'weight': 1,
        'fillOpacity': 0.1  # Transparency for better visualization
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['POP_TOTAL_65+'], 
        aliases=['Total Population 65+']
    )
).add_to(m)

# Display the map in the Notebook
display(m)

# Save the result to a CSV file
edificios[['osm_id', 'SUBSECCAO', 'area', 'POP_65+']].to_csv(
    'file.csv',
    index=False
)

print("Process completed. File saved as pop+64_reconstruido_inteiro.csv")